In [3]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )    


In [128]:
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm
import base64
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [131]:
TIF_PATH = "/data/Marine_picture/Full dataset/Human-annotated-segments/example ISIIS images Gulf of Mexico/"
SAVE_PATH = "/data/Marine_picture/Ground_truth_jpg/"
im_folder = os.listdir(TIF_PATH)
for i in im_folder:
    temp_path = os.path.join(TIF_PATH,i)
    im_path = os.listdir(temp_path)
    count = 0
    for j in tqdm(im_path):
        if j == "Thumbs.db": continue
        count += 1
        name = j.split("_")[0] + "_" + str(count) + ".jpg"
        image = Image.open(os.path.join(temp_path,j))
        jpg_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        cv2.imwrite(SAVE_PATH + name, jpg_image)        

100%|████████████████████████████████████████████████████████████████████████████████████| 362/362 [00:00<00:00, 912.00it/s]


In [141]:
categories={
    0: "chaetognath",
    1: "larval fish",
    2: "Hydromedusa",
    3: "lobate ctenophore",
    4: "Pleurobrachia",
    5: "shrimp",
    6: "Siphonophore",
    7: "stomatopod larva",
    8: "Unknown",
    9: "Thaliac",
    10: "polychaete worm",
    11: "Cumacean",
    12:"ctenophore"
}

In [142]:
possible_cls = ""
for i in categories.keys():
    possible_cls += categories[i] + ","

In [160]:
Result = []
IMAGE_PATH = "/data/Marine_picture/Ground_truth_jpg/"
im_file = os.listdir(IMAGE_PATH)
for i in tqdm(im_file):
    im_path = os.path.join(IMAGE_PATH,i)
    image = encode_image(im_path)
    try:
        response = client.chat.completions.create(
                    model ="GPT-4V",
                messages=[
                    {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "You are an Marine Biologist. I will send you an image that contains zooplanktons. The possible classes are:" + possible_cls + " Please give me the most probable name shown by the given image in the following format: class name. Please just give a short answer and do not provide explanations for your choice."},
                        {
                        "type": "image_url",
                        "image_url": {
                            "url":  f"data:image/jpeg;base64,{image}"
                        },
                        },
                    ],
                    }
                ],
                max_tokens=50
                )
        Result = Result + [[i.split("_")[0],response.choices[0].message.content]]
    except Exception:
        Result = Result + [[i.split("_")[0],"error"]]
        continue

100%|█████████████████████████████████████████████████████████████████████████████████| 2562/2562 [8:54:09<00:00, 12.51s/it]


In [204]:
categories.values()

dict_values(['chaetognath', 'larval fish', 'Hydromedusa', 'lobate ctenophore', 'Pleurobrachia', 'shrimp', 'Siphonophore', 'stomatopod larva', 'Unknown', 'Thaliac', 'polychaete worm', 'Cumacean', 'ctenophore'])

In [164]:
Result2 = Result
for i in Result2:
    if i[0] in categories_2:
        i[0] = categories_2[i[0]]
 

In [212]:
skip = ['polychaete worm','scyphomedusa','Cumacean']
detect_result = {}
detect_result['error'] = 0
for i in Result:
    if i[0] not in skip and i[1] not in skip:
        if i[1] not in categories.values():
            detect_result['error'] += 1
        else:
            if i[1] not in detect_result.keys():
                detect_result[i[1]] = 0
            if i[0] not in skip:
                detect_result[i[1]] += 1
    else:
        print(i)

['larval fish', 'polychaete worm']
['chaetognath', 'polychaete worm']
['scyphomedusa', 'Unknown']
['chaetognath', 'polychaete worm']
['polychaete worm', 'chaetognath']
['scyphomedusa', 'Hydromedusa']
['chaetognath', 'polychaete worm']
['scyphomedusa', 'chaetognath']
['chaetognath', 'polychaete worm']
['scyphomedusa', 'chaetognath']
['Unknown', 'polychaete worm']
['chaetognath', 'polychaete worm']
['scyphomedusa', 'lobate ctenophore']
['larval fish', 'polychaete worm']
['larval fish', 'polychaete worm']
['scyphomedusa', 'chaetognath']
['chaetognath', 'polychaete worm']
['chaetognath', 'polychaete worm']
['Siphonophore', 'polychaete worm']
['chaetognath', 'polychaete worm']
['chaetognath', 'polychaete worm']
['larval fish', 'polychaete worm']
['scyphomedusa', 'Hydromedusa']
['shrimp', 'polychaete worm']
['chaetognath', 'polychaete worm']
['scyphomedusa', 'Hydromedusa']
['scyphomedusa', 'Hydromedusa']
['scyphomedusa', 'Unknown.']
['scyphomedusa', 'Siphonophore']
['scyphomedusa', 'ctenopho

In [211]:
detect_result

{'error': 489,
 'chaetognath': 1066,
 'lobate ctenophore': 85,
 'ctenophore': 83,
 'shrimp': 44,
 'Siphonophore': 42,
 'larval fish': 82,
 'Hydromedusa': 32,
 'stomatopod larva': 33,
 'Unknown': 41}

In [215]:
count = {}
alls = {}
error = 0
for i in Result2:
    if i[0] not in skip and i[1] not in skip:
        if i[0] not in count.keys():
            count[i[0]] = 0
            alls[i[0]] = 0
        alls[i[0]] += 1
        if i[0] == i[1]:
            count[i[0]] += 1
        if i[1] not in categories.values():
            error += 1
print(count)

{'larval fish': 62, 'Thaliac': 0, 'chaetognath': 374, 'Siphonophore': 5, 'stomatopod larva': 13, 'shrimp': 7, 'Hydromedusa': 23, 'lobate ctenophore': 13, 'Unknown': 0, 'Pleurobrachia': 0, 'cteno': 0}


In [216]:
count

{'larval fish': 62,
 'Thaliac': 0,
 'chaetognath': 374,
 'Siphonophore': 5,
 'stomatopod larva': 13,
 'shrimp': 7,
 'Hydromedusa': 23,
 'lobate ctenophore': 13,
 'Unknown': 0,
 'Pleurobrachia': 0,
 'cteno': 0}

In [217]:
alls

{'larval fish': 704,
 'Thaliac': 16,
 'chaetognath': 508,
 'Siphonophore': 98,
 'stomatopod larva': 118,
 'shrimp': 306,
 'Hydromedusa': 158,
 'lobate ctenophore': 18,
 'Unknown': 18,
 'Pleurobrachia': 44,
 'cteno': 9}

In [185]:
temp

2562

In [182]:
error

558

In [163]:
categories_2={
    'chaeto': "chaetognath",
    'fish': "larval fish",
    'Hydro': "Hydromedusa",
    'LC': "lobate ctenophore",
    'Pleuro': "Pleurobrachia",
    5: "shrimp",
    'siphon': "Siphonophore",
    'stomato': "stomatopod larva",
    'Unk': "Unknown",
    9: "Thaliac",
    'poly': "polychaete worm",
    11: "Cumacean",
    12:"ctenophore"
}

In [161]:
#np.save("/data/Marine_picture/GPT4_result",Result)

In [149]:
[] + [["response1","response2"]]

[['response1', 'response2']]

In [151]:
type(response.choices[0].message.content)

str

In [ ]:
response = client.chat.completions.create(
            model ="GPT-4V",
        messages=[
            {
            "role": "user",
            "content": [
                {"type": "text", "text": "You are an Marine Biologist. I will send you an image that contains planktons. The possible classes are:" + possible_cls +" Please give me the most probable name and the bounding box of the planktons shown by the given image in the following format: CLASS NAME [TOP LEFT COORDINATE, BOTTOM RIGHT COORDINATE]. Please just give a short answer and do not provide explanations for your choice."},
                {
                "type": "image_url",
                "image_url": {
                    "url":  f"data:image/jpeg;base64,{image}"
                },
                },
            ],
            }
        ],
        max_tokens=50
        )